In [ ]:
import pandas as pd
import numpy as np
import json
import glob

In [ ]:
def payloads_from_json_log_single(filename):
    with open(filename, 'r') as f:
        for line in f:
            obj = json.loads(line)
            yield {
                "device_id": obj["end_device_ids"]["device_id"],
                "received_at": obj["received_at"],
                "gateway_id": [m["gateway_ids"]["gateway_id"] for m in obj["uplink_message"]["rx_metadata"]],
                **obj["uplink_message"].get("decoded_payload", {})
            } 
            

def payloads_from_json_log(fname_pattern):
    for fname in glob.glob(fname_pattern):
        yield from payloads_from_json_log_single(fname)

In [ ]:
df = pd.DataFrame(payloads_from_json_log("/storage/stations/GEODOS/*.json"))
df["received_at"] = pd.to_datetime(df["received_at"])

In [ ]:
df

In [ ]:
# iterate over unique device IDs seen in logged messages
for dev_id in np.unique(df["device_id"]):
    # select rows from that particular device
    df_filtered = df[df["device_id"] == dev_id]
    
    # some plots...
    df_filtered.plot("received_at", "V", figsize=(25, 5), title=dev_id)
    df_filtered.plot("received_at", "°C", figsize=(25, 5), title=dev_id)